In [1]:
import pandas as pd
import lightgbm
import category_encoders as ce
from sklearn.preprocessing import CategoricalEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import numpy as np
import pandas as pd

#random seeds for stochastic parts of neural network 
np.random.seed(10)
from tensorflow import set_random_seed
set_random_seed(15)

from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout
from keras.layers.embeddings import Embedding

from sklearn.model_selection import StratifiedKFold

import datetime

Using TensorFlow backend.


In [88]:
df_train = pd.read_csv("pre_pro_train.csv")
df_test = pd.read_csv("pre_pro_test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [70]:
list(set(df_train.columns) - set(cat_cols) - set(numeric_cols))

['booking_dayofweek',
 'actual_days_booked',
 'checkout_dayofweek',
 'n_weekend_days',
 'checkout_day',
 'booking_day',
 'checkin_dayofweek',
 'checkout_month',
 'booking_month',
 'checkin_day',
 'person_per_room',
 'checkin_year',
 'n_rooms_booked',
 'checkout_year',
 'checkin_month',
 'amount_spent_per_room_night_scaled',
 'booking_year']

In [60]:
# df_train.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)
# df_test.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)


In [22]:
# df_train.fillna(0,inplace=True,axis=0)
# df_test.fillna(0,inplace=True,axis=0)

In [89]:
cat_cols = ['channel_code','main_product_code','persontravellingid','resort_region_code',\
      'resort_type_code','room_type_booked_code','season_holidayed_code',\
       'state_code_residence','state_code_resort','member_age_buckets','booking_type_code',\
       'cluster_code','reservationstatusid_code','resort_id','season_holidayed_code_is_na',\
        'ses', 'state_code_residence_is_na'    
      ]

In [90]:
numeric_cols = ['numberofadults','numberofchildren','roomnights','total_pax',\
               'booking_dayofweek',
 'actual_days_booked',
 'checkout_dayofweek',
 'n_weekend_days',
 'checkout_day',
 'booking_day',
 'checkin_dayofweek',
 'checkout_month',
 'booking_month',
 'checkin_day',
 'person_per_room',
 'checkin_year',
 'n_rooms_booked',
 'checkout_year',
 'checkin_month',
 'booking_year']
len(numeric_cols)

20

In [91]:
cols_use = cat_cols + numeric_cols
len(cols_use)

37

In [92]:

X_train, y_train = df_train[cols_use], df_train['amount_spent_per_room_night_scaled']
X_test = df_test[cols_use]
X_train.shape

(341424, 37)

In [93]:
X_train.head()

,channel_code,main_product_code,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,season_holidayed_code,state_code_residence,state_code_resort,member_age_buckets,...,checkin_dayofweek,checkout_month,booking_month,checkin_day,person_per_room,checkin_year,n_rooms_booked,checkout_year,checkin_month,booking_year
0,3,1,46,3,3,3,2.0,7.0,3,1,...,3,4,4,5,2.0,2018,1.0,2018,4,2018
1,1,1,46,3,3,4,2.0,7.0,5,1,...,5,4,1,11,2.0,2015,1.0,2015,4,2015
2,1,1,47,1,5,4,2.0,7.0,1,1,...,6,2,1,1,2.0,2015,1.0,2015,2,2015
3,1,1,46,2,2,3,2.0,7.0,2,1,...,3,6,5,11,4.0,2015,1.0,2015,6,2015
4,1,1,46,2,2,4,2.0,7.0,2,1,...,0,12,9,14,2.0,2015,1.0,2015,12,2015


In [94]:
encoder = OrdinalEncoder(dtype=np.int32,)
X_train[cat_cols] = encoder.fit_transform(
    X_train[cat_cols]
)
X_train.shape

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(341424, 37)

In [95]:
encoder = OrdinalEncoder(dtype=np.int32,)
X_test[cat_cols] = encoder.fit_transform(
    X_test[cat_cols]
)
X_test.shape

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(146765, 37)

In [30]:
X_train.head()

,channel_code,main_product_code,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,season_holidayed_code,state_code_residence,state_code_resort,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,numberofadults,numberofchildren,roomnights,total_pax
0,2,0,1,2,3,2,2,7,2,5,0,5,2,7,2,0,1,3
1,0,0,1,2,3,3,2,7,4,5,0,5,0,1,2,0,5,2
2,0,0,2,0,5,3,2,7,0,5,0,4,0,9,2,0,4,2
3,0,0,1,1,2,2,2,7,1,5,0,3,0,25,2,2,5,2
4,0,0,1,1,2,3,2,7,1,5,0,3,0,25,2,0,5,2


In [96]:
col_vals_dict = {c: list(X_train[c].unique()) for c in cat_cols}

In [97]:
col_vals_dict

{'channel_code': [2, 0, 1],
 'main_product_code': [0, 1, 2, 3, 4],
 'persontravellingid': [1, 2, 0, 3, 4, 5],
 'resort_region_code': [2, 0, 1],
 'resort_type_code': [3, 5, 2, 1, 6, 4, 0],
 'room_type_booked_code': [2, 3, 1, 0, 4, 5],
 'season_holidayed_code': [2, 3, 4, 1, 0],
 'state_code_residence': [7,
  8,
  2,
  5,
  6,
  3,
  4,
  1,
  9,
  14,
  0,
  24,
  15,
  12,
  10,
  27,
  23,
  22,
  19,
  11,
  13,
  20,
  17,
  16,
  25,
  26,
  21,
  18,
  30,
  33,
  31,
  29,
  32,
  28,
  36,
  34,
  35,
  37],
 'state_code_resort': [2, 4, 0, 1, 5, 7, 8, 3, 9, 6, 10],
 'member_age_buckets': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'booking_type_code': [0, 1],
 'cluster_code': [0, 1, 2, 3, 4, 5],
 'reservationstatusid_code': [0, 1, 2, 3],
 'resort_id': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31],
 'season_holidayed_code_is_na': [0, 1],
 'ses': [0, 1, 2, 3],

In [98]:
embed_cols = []
for c in col_vals_dict:
    if len(col_vals_dict[c])>1:
        embed_cols.append(c)
        print(c + ': %d values' % len(col_vals_dict[c])) #look at value counts to know the embedding dimensions

channel_code: 3 values
main_product_code: 5 values
persontravellingid: 6 values
resort_region_code: 3 values
resort_type_code: 7 values
room_type_booked_code: 6 values
season_holidayed_code: 5 values
state_code_residence: 38 values
state_code_resort: 11 values
member_age_buckets: 10 values
booking_type_code: 2 values
cluster_code: 6 values
reservationstatusid_code: 4 values
resort_id: 32 values
season_holidayed_code_is_na: 2 values
ses: 4 values
state_code_residence_is_na: 2 values


In [109]:
# ps_ind_02_cat: 5 values
# ps_ind_04_cat: 3 values
# ps_ind_05_cat: 8 values
# ps_car_01_cat: 13 values
# ps_car_02_cat: 3 values
# ps_car_03_cat: 3 values
# ps_car_04_cat: 10 values
# ps_car_05_cat: 3 values
# ps_car_06_cat: 18 values
# ps_car_07_cat: 3 values
# ps_car_09_cat: 6 values
# ps_car_10_cat: 3 values
# ps_car_11_cat: 104 values

# channel_code: 3 values
# main_product_code: 5 values
# persontravellingid: 6 values
# resort_region_code: 3 values
# resort_type_code: 7 values
# room_type_booked_code: 6 values
# season_holidayed_code: 5 values
# state_code_residence: 38 values
# state_code_resort: 11 values
# member_age_buckets: 10 values
# booking_type_code: 2 values
# cluster_code: 6 values
# reservationstatusid_code: 4 values
# resort_id: 32 values
# season_holidayed_code_is_na: 2 values
# ses: 4 values
# state_code_residence_is_na: 2 values


#def build_embedding_network():

inputs = []
embeddings = []

input_ps_ind_02_cat = Input(shape=(1,))
embedding = Embedding(3, 2, input_length=1)(input_ps_ind_02_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_ind_02_cat)
embeddings.append(embedding)

input_ps_ind_04_cat = Input(shape=(1,))
embedding = Embedding(5, 3, input_length=1)(input_ps_ind_04_cat)
embedding = Reshape(target_shape=(3,))(embedding)
inputs.append(input_ps_ind_04_cat)
embeddings.append(embedding)

input_ps_ind_05_cat = Input(shape=(1,))
embedding = Embedding(6, 4, input_length=1)(input_ps_ind_05_cat)
embedding = Reshape(target_shape=(4,))(embedding)
inputs.append(input_ps_ind_05_cat)
embeddings.append(embedding)

input_ps_car_01_cat = Input(shape=(1,))
embedding = Embedding(3, 2, input_length=1)(input_ps_car_01_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_car_01_cat)
embeddings.append(embedding)

input_ps_car_02_cat = Input(shape=(1,))
embedding = Embedding(7, 5, input_length=1)(input_ps_car_02_cat)
embedding = Reshape(target_shape=(5,))(embedding)
inputs.append(input_ps_car_02_cat)
embeddings.append(embedding)

input_ps_car_03_cat = Input(shape=(1,))
embedding = Embedding(6, 4, input_length=1)(input_ps_car_03_cat)
embedding = Reshape(target_shape=(4,))(embedding)
inputs.append(input_ps_car_03_cat)
embeddings.append(embedding)


input_ps_car_04_cat = Input(shape=(1,))
embedding = Embedding(5, 3, input_length=1)(input_ps_car_04_cat)
embedding = Reshape(target_shape=(3,))(embedding)
inputs.append(input_ps_car_04_cat)
embeddings.append(embedding)

print('cc')
input_ps_car_05_cat = Input(shape=(1,))
embedding = Embedding(38, 14, input_length=1)(input_ps_car_05_cat)
embedding = Reshape(target_shape=(14,))(embedding)
inputs.append(input_ps_car_05_cat)
embeddings.append(embedding)

print('cc1')
input_ps_car_06_cat = Input(shape=(1,))
embedding = Embedding(11, 5, input_length=1)(input_ps_car_06_cat)
embedding = Reshape(target_shape=(5,))(embedding)
inputs.append(input_ps_car_06_cat)
embeddings.append(embedding)
print('cc3')
input_ps_car_07_cat = Input(shape=(1,))
embedding = Embedding(10, 3, input_length=1)(input_ps_car_07_cat)
embedding = Reshape(target_shape=(3,))(embedding)
inputs.append(input_ps_car_07_cat)
embeddings.append(embedding)
print('cc4')


input_ps_car_017_cat = Input(shape=(1,))
embedding = Embedding(2, 2, input_length=1)(input_ps_car_017_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_car_017_cat)
embeddings.append(embedding)




# input_ps_car_07_cat = Input(shape=(1,))
# embedding = Embedding(10, 3, input_length=1)(input_ps_car_07_cat)
# embedding = Reshape(target_shape=(3,))(embedding)
# inputs.append(input_ps_car_07_cat)
# embeddings.append(embedding)

input_ps_car_09_cat = Input(shape=(1,))
embedding = Embedding(6, 3, input_length=1)(input_ps_car_09_cat)
embedding = Reshape(target_shape=(3,))(embedding)
inputs.append(input_ps_car_09_cat)
embeddings.append(embedding)
print('cc5')
input_ps_car_10_cat = Input(shape=(1,))
embedding = Embedding(4, 2, input_length=1)(input_ps_car_10_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_car_10_cat)
embeddings.append(embedding)
print('cc6')
input_ps_car_11_cat = Input(shape=(1,))
embedding = Embedding(32, 8, input_length=1)(input_ps_car_11_cat)
embedding = Reshape(target_shape=(8,))(embedding)
inputs.append(input_ps_car_11_cat)
embeddings.append(embedding)




input_ps_car_12_cat = Input(shape=(1,))
embedding = Embedding(2, 2, input_length=1)(input_ps_car_12_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_car_12_cat)
embeddings.append(embedding)


input_ps_car_13_cat = Input(shape=(1,))
embedding = Embedding(4, 3, input_length=1)(input_ps_car_13_cat)
embedding = Reshape(target_shape=(3,))(embedding)
inputs.append(input_ps_car_13_cat)
embeddings.append(embedding)


input_ps_car_14_cat = Input(shape=(1,))
embedding = Embedding(2, 2, input_length=1)(input_ps_car_14_cat)
embedding = Reshape(target_shape=(2,))(embedding)
inputs.append(input_ps_car_14_cat)
embeddings.append(embedding)

print('cc7')
input_numeric = Input(shape=(20,))
embedding_numeric = Dense(16)(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)
print('cc8')
x = Concatenate()(embeddings)
x = Dense(512, activation='relu')(x)
x = Dense(216, activation='relu')(x)
x = Dense(64, activation='relu')(x)

output = Dense(1)(x)

model = Model(inputs, output)
model.compile(loss='mean_squared_error', optimizer='adam')


cc
cc1
cc3
cc4
cc5
cc6
cc7
cc8


In [110]:
def preproc(X_train, X_val, X_test):

    input_list_train = []
    input_list_val = []
    input_list_test = []
    
    #the cols to be embedded: rescaling to range [0, # values)
    for c in embed_cols:
        raw_vals = np.unique(X_train[c])
        val_map = {}
        for i in range(len(raw_vals)):
            val_map[raw_vals[i]] = i       
        input_list_train.append(X_train[c].map(val_map).values)
        input_list_val.append(X_val[c].map(val_map).fillna(0).values)
        input_list_test.append(X_test[c].map(val_map).fillna(0).values)
     
    #the rest of the columns
    other_cols = [c for c in X_train.columns if (not c in embed_cols)]
    input_list_train.append(X_train[other_cols].values)
    input_list_val.append(X_val[other_cols].values)
    input_list_test.append(X_test[other_cols].values)
    
    return input_list_train, input_list_val, input_list_test

In [111]:
from sklearn.model_selection import KFold
#K = 8
#runs_per_fold = 3
n_epochs = 30

#cv_ginis = []
#full_val_preds = np.zeros(np.shape(X_train)[0])
#y_preds = np.zeros((np.shape(X_test)[0],K))

#kfold = KFold(n_splits = K, 
                            #random_state = 231, 
                            #shuffle = True)    

#for i, (f_ind, outf_ind) in enumerate(kfold.split(X_train, y_train)):

X_train_f, X_val_f = X_train, X_train
y_train_f, y_val_f = y_train, y_train
    
X_test_f = X_test.copy()
    
    #upsampling adapted from kernel: 
    #https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283
    #pos = (pd.Series(y_train_f == 1))
    
    # Add positive examples
    #X_train_f = pd.concat([X_train_f, X_train_f.loc[pos]], axis=0)
    #y_train_f = pd.concat([y_train_f, y_train_f.loc[pos]], axis=0)
    
    # Shuffle data
idx = np.arange(len(X_train_f))
np.random.shuffle(idx)
X_train_f = X_train_f.iloc[idx]
y_train_f = y_train_f.iloc[idx]
    
    #preprocessing
proc_X_train_f, proc_X_val_f, proc_X_test_f = preproc(X_train_f, X_val_f, X_test_f)

# #track oof prediction for cv scores
# val_preds = 0
# #for j in range(runs_per_fold):
# #break

model.fit(proc_X_train_f, y_train_f.values, epochs=10, batch_size=4096, verbose=1)

        #val_preds += NN.predict(proc_X_val_f)[:,0] / runs_per_fold
        #y_preds[:,i] += NN.predict(proc_X_test_f)[:,0] / runs_per_fold
        
    #full_val_preds[outf_ind] += val_preds
        
    #cv_gini = gini_normalizedc(y_val_f.values, val_preds)
    #cv_ginis.append(cv_gini)
    #print ('\nFold %i prediction cv gini: %.5f\n' %(i,cv_gini))


Epoch 1/10
341424/341424 [==============================] - 7s 22us/step - loss: 1408.3785
Epoch 2/10
341424/341424 [==============================] - 6s 16us/step - loss: 1.9645
Epoch 3/10
341424/341424 [==============================] - 6s 17us/step - loss: 1.3627
Epoch 4/10
341424/341424 [==============================] - 6s 16us/step - loss: 1.3503
Epoch 5/10
341424/341424 [==============================] - 6s 16us/step - loss: 1.3450
Epoch 6/10
341424/341424 [==============================] - 6s 18us/step - loss: 1.3402
Epoch 7/10
341424/341424 [==============================] - 6s 18us/step - loss: 1.3242
Epoch 8/10
341424/341424 [==============================] - 6s 19us/step - loss: 1.3012
Epoch 9/10
341424/341424 [==============================] - 6s 18us/step - loss: 1.2782
Epoch 10/10
341424/341424 [==============================] - 6s 19us/step - loss: 1.2587


In [112]:
l = model.predict(proc_X_test_f)

In [113]:
sample_submission['amount_spent_per_room_night_scaled'] = l
sample_submission.to_csv(
    "neural_net_fet_added.csv",
    index=False)

In [ ]:
len(cat_cols)

In [108]:
len(proc_X_train_f)

18

In [43]:
df_test.shape

(146765, 18)